In [7]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
from pyswarm import pso

# General Functions

In [ ]:
'''Function that prints all required data of a graph G.
(Plots network graph and vertex degree distribution histogram, then prints |V|,|E|, 
average vertex degree, CC, number of triangles, average path length and diameter.)'''
def print_network_data(G):
    v_degrees = np.array(list(dict(nx.degree(G)).values()))
    nx.draw(G, node_size = 2.5 * v_degrees, node_color = 'blue',
    alpha = 0.2, edge_color = 'green')
    plt.show()

    vertex_degrees = list(dict(nx.degree(G)).values()) 
    plt.hist(vertex_degrees, bins = np.linspace(0, 300, 80),
            facecolor='blue', alpha=0.75, rwidth = 0.9) 
    plt.title("Vertex degree distribution") 
    plt.grid(True)
    plt.show()
    
    n = nx.number_of_nodes(G)
    m = nx.number_of_edges(G)
    cc = nx.average_clustering(G)
    triangles = np.trace(np.matrix((nx.to_numpy_matrix(G)**3))/6)
    try:
        avg_path_length = nx.average_shortest_path_length(G)
        diameter = nx.diameter(G)
    except: #in case graph is not fully connected
        avg_path_length = 'error'
        diameter = 'error'
    print("|V| =", n)
    print("|E| =", m)
    print("Average degree is", 2 * m / n) 
    print("CC =", cc)
    print('Number of triangles = ', triangles)
    print('Average path length = ', avg_path_length)
    print('Diameter = ', diameter)
    return None

# Code For Facebook Network

In [ ]:
'''Retrieving Facebook network data and plotting it'''
FbData = np.loadtxt("facebook_combined.txt")
FbData = FbData.astype(int)
F = nx.Graph()
F.add_edges_from(FbData)
print_network_data(F)

/Users/George/Desktop/anaconda/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:126: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  b = plt.ishold()


# Code For In-Built Random Models (ER,WS,BA)

In [ ]:
'''Creating and plotting the Erdős-Rényi network'''
E = nx.erdos_renyi_graph(4039,0.01081)
print_network_data(E)

'''Creating and plotting the Watts-Strogatz Network'''
W = nx.watts_strogatz_graph(4039, 44, 0.05)
print_network_data(W)

'''Creating and plotting the Barabási-Albert network'''
B = nx.barabasi_albert_graph(4039, 22)
print_network_data(B)

# Code For MDA Random Model

In [ ]:
'''Function for creating a MDA network graph'''
def mda(n,m):
    A = np.ones((m+1,m+1)) - np.diag(np.ones(m+1))
    B = np.zeros((n,n)) 
    B[0:m+1,0:m+1] = A 
    for i in range(m+1,n):
        mediator=np.random.choice(np.array(list(range(i))))
        neighbor = B[mediator,:]
        neighbor_index = np.where(neighbor >0)[0]
        vec_rand = np.random.choice(np.size(neighbor_index),m,replace = False) 
        a= neighbor_index[vec_rand[range(m)]]
        B[a,i]=1
        B[i,a]=1
    M = nx.Graph(B)
    return M
    
'''Plotting the Mediation-Driven Attachment Network'''
M = mda(4039,22)
print_network_data(M)

# Code For Clustom Random Model

In [ ]:
'''Function that splits the number of nodes, n, into g clusters in the main function'''
def splitn(n,g):
    listofsubgraphs = [1]*(g+1) 
    #extra cluster because 0th cluster is too small to be significant
    for _ in range(n-g):
        listofsubgraphs[int(np.cbrt(np.random.randint((g+1)**(3))))] += 1
        #using y=x**3 to form a biased distribution
    for i in range(len(listofsubgraphs)-2,-1,-2):
        listofsubgraphs += [listofsubgraphs.pop(i)] 
        #sort to get largest subgraphs to be in the middle
    return listofsubgraphs

'''Function for creating WS graphs' adjacency matrices in the main function.'''
def wsmatrix(n, l, p):
    k = int(n*(l**(n//500+1))/2)*2
    B = (np.arange(n).reshape(n, 1) - np.arange(n).reshape(1, n)) % n
    A = 1 * (((B >= 1) & (B <= k/2)) | ((B >= n - k/2) & (B <= n-1)))
    for i in range(n):
        v_maybe_rewired = (i + np.arange(1, k //2 +1)) % n
        v_rewired = v_maybe_rewired[np.random.rand(k // 2) < p]
        v_maybe_rewired_to = np.where((A[i, :] == 0) & (np.arange(n) != i))[0]
        v_rewired_to = np.random.choice(v_maybe_rewired_to, len(v_rewired), replace = False)
        A[v_rewired, i] = 0
        A[i, v_rewired] = 0
        A[v_rewired_to, i] = 1
        A[i, v_rewired_to] = 1
    return A

'''Function for creating Clustom network graph'''
def customrandomgraph(n, g, l, p, q):
    if n < g or p > 1 or q > 1:
        return 'error'
    else: n = splitn(n,g)
    ws = [wsmatrix(i,l,p) for i in n]
    listofclusters = [[] for i in range(g+1)]
    for h in range((g+1)**2):
        i = h//(g+1)
        j = h%(g+1)
        if j == i:
            listofclusters[i] += [ws[i]]
        else:   
            distance = min(j-i,g-j+i+1)
            listofclusters[i] += [1*(np.random.rand(n[i],n[j]) < (q/distance**2))]
    A = np.triu(np.concatenate([np.concatenate(m,1) for m in listofclusters]))
    custommatrix = np.mat(A + A.T)
    return nx.Graph(custommatrix) 

'''Function for plotting the Clustom network
m is the desired number of edges, e is the error range for number of edges
Prints progress as it runs'''
def plotcrm(n, g, l, p, q, m, e):
    difference = e
    while difference >= e:
        print('Finding graph,')
        try:
            print('creating adjacency matrix,')
            C = customrandomgraph(n, g, l, p, q)
            print('checking connectedness,')
            d = nx.diameter(C)
        except:
            print('error.')
            continue
        else:
            difference = abs(nx.number_of_edges(C) - m)
            if difference >= e:
                print('number of edges not optimal.')
    print('graph found, plotting graph.')
    print_network_data(C)
    return None

'''Plotting the Clustom Network'''
plotcrm(4039, 7, 0.27, 0.09, 0.0001, 88234, 3000)

# Code For SIR

In [ ]:
'''Generating network for SIR model'''
G = plotcrm(400, 7, 0.27, 0.09, 0.001,0,99999)

'''SIR function'''
def SIR(A, init, p, q, d):
    #A: adjancency matrix
    #init: number of infected nodes at day 0
    #p: probabilty of infecting those who are Susceptible
    #q: probability of recovering from being Infected
    #d: number of days
    
    G = nx.Graph(A)
    nx.draw_networkx(G)
    plt.show()
    n = len(A)
    Number_of_susceptible = [n-init]
    Number_of_infected = [init]
    Number_of_recovered = [0]
    print('Day 0')
    
    #create a list to contain nodes that are infected
    #initialise with random nodes that are Infected
    IList = np.array(random.sample(range(n),init)) 
    #create a list to contain nodes that have a chance of being infected
    SList = np.setdiff1d(np.arange(n),IList) #contain nodes that has not been Infected
    RList = np.array([]).astype(int) #initialise with empty list

    print('Number of Infected',init,', Infected List:',IList, '\n')
    
    #as the days go by from day 0 to day n+1, the lists will change, 
    so we use a for loop to iterate each day
    for i in range(1,d): 
        print("Day",i)
        #search through the infected list to check if each node has recovered
        for node in IList: 
            if np.random.rand() < q:
                RList = np.append(RList,node) #add the recovered nodes to the Recovered List
        IList = np.setdiff1d(IList,RList) #remove recovered nodes from the infected list       
        
        #finding infected neighbours of susceptible nodes
        B = np.copy(A)
        B[:,SList.astype(int)] = 0  
        B[:,RList.astype(int)] = 0
        B[IList.astype(int),:] = 0
        B[RList.astype(int),:] = 0
        #the probability of infection for each susceptible node
        is based on the number of infected neighbours
        #loop through the susceptible list and append nodes that
        have been infected to the infected list
        for node in SList:
            #the probability of infection for each susceptible 
            node is based on the number of infected neighbours
            Probability_of_infection = 1-(1-p)**(np.sum(B[node]))
            if np.random.rand() < Probability_of_infection:
                #add the newly infected nodes to the Infected list
                IList = np.append(IList,node) 
        #remove the infected nodes from the Susceptible list
        SList = np.setdiff1d(SList,IList) 
        Number_of_susceptible.append(len(SList))
        Number_of_infected.append(len(IList))
        Number_of_recovered.append(len(RList))
        #print("Susceptible:", len(SList), ", Infected:", len(IList), ", Recovered List:", len(RList))
        #print()

    #plt.plot(np.arange(d), Number_of_susceptible, label = 'Susceptible')
    #plt.plot(np.arange(d), Number_of_infected, label = 'Infected')
    #plt.plot(np.arange(d), Number_of_recovered, label = 'Recovered')
    #plt.legend(loc = 'center right')
    #plt.xlabel('Days')
    #plt.ylabel('Individuals')
    #plt.show()
    return Number_of_infected

'''Running SIR'''
RunSIR = SIR(A, 8, 0.6, 0.6, 12)

# Code For PSO

In [ ]:
'''Retrieving Google Trend data'''
Googletrend_KimTrumpData = np.loadtxt("TrumpKim12weeks.txt")
days = list(range(12))
google_search = Googletrend_KimTrumpData
A = (nx.adjacency_matrix(G).todense())

'''Modified SIR function to reduce computation time in PSO function'''
def PSO_SIR(A, init, p, q, w):
    G = nx.Graph(A)
    n = len(A)
    infected = [init]
    
    IList = np.arange(0,n,int(n/init)) #systematic sampling to partially seed results
    SList = np.setdiff1d(np.arange(n),IList)
    RList = np.array([]).astype(int)
    
    for i in range(1,w): 
        for node in IList: 
            if np.random.rand() < q:
                RList = np.append(RList,node)
        IList = np.setdiff1d(IList,RList)    
        
        B = np.copy(A)
        B[:,SList.astype(int)] = 0  
        B[:,RList.astype(int)] = 0
        B[IList.astype(int),:] = 0
        B[RList.astype(int),:] = 0
        
        for node in SList:
            Probability_of_infection = 1-(1-p)**(np.sum(B[node]))
            if np.random.rand() < Probability_of_infection:
                IList = np.append(IList,node) 
        SList = np.setdiff1d(SList,IList)
        infected.append(len(IList))
    
    maxvalue = max(infected)
    normalised_infected = np.array(infected)*(100/maxvalue)
    trend = Googletrend_KimTrumpData
    difference = np.sqrt(sum(((normalised_infected-trend)/100)**2))
    #each value is taken as a percentage to keep difference small
    print('p = %f, q = %f, difference = %f'%(p, q, difference))
    print(normalised_infected)
    return difference

'''Function used in the PSO function'''
def f(x): #x = [p,q]
    p = x[0]
    q = x[1]
    difference = PSO_SIR(A, 8, p , q, 12)
    return difference

'''Running PSO'''
lb = [0.2, 0.5]
ub = [0.4, 0.7]
pq_value, distance = pso(f, lb, ub) 
p = pq_value[0]
q = pq_value[1]

'''Code for printing the Google Trend and optimized SIR trend'''
distance = 1
while distance >= 0.6:
    plt.title('Google Trend v.s. SIR Trend')
    plt.plot(days, google_search, color='g', label = 'Google Trend')
    plt.legend(loc = 'center right')
    distance = PSO_SIR(A, 8, p, q, 12)